http://nilc.icmc.usp.br/nilc/index.php/repositorio-de-word-embeddings-do-nilc


In [ ]:
!wget http://143.107.183.175:22980/download.php?file=embeddings/word2vec/cbow_s300.zip

In [ ]:
!pip install -U gensim scipy

In [ ]:
!unzip /content/download.php?file=embeddings%2Fword2vec%2Fcbow_s300.zip

In [ ]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('cbow_s300.txt')

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/datasetTeste_Final.csv")

In [ ]:
df

In [ ]:
from re import sub

def text_to_word_list(text):
    ''' Pre process and convert texts to a list of words 
    method inspired by method from eliorc github repo: https://github.com/eliorc/Medium/blob/master/MaLSTM.ipynb'''
    #text = remove_polish_letters(text)
    text = str(text)
    text = text.lower()

    # Clean the text
    text = sub(r"[\d,!?.\/'+]", " ", text)
    text = sub(r"\+", " mais ", text)
    text = sub(r",", " ", text)
    text = sub(r"\.", " ", text)
    text = sub(r"!", " ! ", text)
    text = sub(r"\?", " ? ", text)
    text = sub(r"'", " ", text)
    text = sub(r":", " : ", text)
    text = sub(r"\s{2,}", " ", text)

    text = text.split()

    return text  

In [ ]:
df.headlines = df.headlines.apply(lambda x: text_to_word_list(x))

In [ ]:
file_model = df.copy()
file_model = file_model[file_model.headlines.str.len()>1]

In [ ]:
from gensim.models.phrases import Phrases, Phraser

sent = [row for row in file_model.headlines]
phrases = Phrases(sent, min_count=1, progress_per=50000)
bigram = Phraser(phrases)
sentences = bigram[sent]
sentences[1]

In [ ]:
from gensim.models import Word2Vec
import multiprocessing
from time import time

w2v_model = Word2Vec(min_count=3,
                     window=4,
                     vector_size=300,
                     sample=1e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=multiprocessing.cpu_count()-1)

start = time()

w2v_model.build_vocab(sentences, progress_per=50000)

print('Time to build vocab: {} mins'.format(round((time() - start) / 60, 2)))

In [ ]:
start = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - start) / 60, 2)))

w2v_model.init_sims(replace=True)

In [ ]:
w2v_model.save("word2vec.model")

In [ ]:
file_export = file_model.copy()
file_export['old_headlines'] = file_export.headlines
file_export.old_headlines = file_export.old_headlines.str.join(' ')
file_export.headlines = file_export.headlines.apply(lambda x: ' '.join(bigram[x]))
#file_export.compound = file_export.compound.astype('int8')

In [ ]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.cluster import KMeans

In [ ]:
word_vectors = Word2Vec.load("/content/word2vec.model").wv

In [ ]:
model = KMeans(n_clusters=3, max_iter=1000, random_state=True, n_init=50).fit(X=word_vectors.vectors.astype('double'))

In [ ]:
word_vectors.similar_by_vector("bolsonaro", topn=10, restrict_vocab=None)

In [ ]:
positive_cluster_index = 1
positive_cluster_center = model.cluster_centers_[positive_cluster_index]
negative_cluster_center = model.cluster_centers_[1-positive_cluster_index]

In [ ]:
words = pd.DataFrame(list(word_vectors.index_to_key))
words.columns = ['words']
words['vectors'] = words.words.apply(lambda x: word_vectors[f'{x}'])
words['cluster'] = words.vectors.apply(lambda x: model.predict([np.array(x)]))
words.cluster = words.cluster.apply(lambda x: x[0])

In [ ]:
words['cluster_value'] = [1 if i==positive_cluster_index else -1 for i in words.cluster]
words['closeness_score'] = words.apply(lambda x: 1/(model.transform([x.vectors]).min()), axis=1)
words['sentiment_coeff'] = words.closeness_score * words.cluster_value

In [ ]:
words